In [1]:
import os
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import selenium.webdriver.support.ui as ui
import selenium.webdriver.support.expected_conditions as EC
import time
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
#could not use requests as requests does not have a JavaScript engine
#setting up webdriver
ta = 'https://www.tripadvisor.com.sg/Restaurants-g294265-Singapore.html'

driver = webdriver.Chrome(executable_path='Downloads/chromedriver')
driver.get(ta)

In [3]:
#good soup!
soup = bs(driver.page_source, 'html.parser')

In [4]:
#create dataframe
column_names = ["RestaurantName", "Review", "TARanking", "NameLength", "Avg$PerPax", "NumOfReviews", "Photos", "ZipCode", "Claimed", "Michelin", "TravellersChoice", "ReserveOnTA"]

df = pd.DataFrame(columns=column_names)
df

,RestaurantName,Review,TARanking,NameLength,Avg$PerPax,NumOfReviews,Photos,ZipCode,Claimed,Michelin,TravellersChoice,ReserveOnTA


In [5]:
while len(df.index) <= 500:
    hrefs = soup.find_all(class_='OhCyu')
    i = 0
    while i < len(hrefs):
        #selenium shtuff
        tripadvisorhttp = 'https://www.tripadvisor.com.sg'
        rurl = tripadvisorhttp + hrefs[i].find(href=True)['href']
        driver.get(rurl)
        rsoup = bs(driver.page_source, 'html.parser')
        
        #name of restaurant
        try:
            name = rsoup.find(class_='fHibz').text
        except:
            name = None
        
        #how many previous reviews?
        try:
            prev_reviews = int(rsoup.find(class_='eBTWs').text.split(' ')[0].replace(',',''))
        except:
            prev_reviews = 0
        
        #trip advisor's ranking
        try:
            TAranking = driver.find_elements_by_xpath('.//span[@class = "dyeJW dUpPX"]')[0].text.split("#")[-1].split(" ")[0]
            if "," in list(TAranking):
                TArankinglist = TAranking.split(",")
                TAranking = ""
                for item in TArankinglist:
                    TAranking += item
                TAranking = int(TAranking)
            else:
                TAranking = int(TAranking)
        except:
            TAranking = 0
            
        #the trip advisor review
        try:
            revbubbles = str(rsoup.find(class_="iPqaD _F G- ddFHE eKwUx"))[str(rsoup.find(class_="iPqaD _F G- ddFHE eKwUx")).find('''aria-label="''')+12:str(rsoup.find(class_="iPqaD _F G- ddFHE eKwUx")).find(''' of 5 bubbles"''')+13]
            review = float(str(revbubbles).split(" of")[0])
        except:
            review = None
        
        #only looking at restaurants with more than 5 reviews and a top 1000 ranking
        if name not in df.values and name != None and prev_reviews > 5 and review != None and TAranking <= 1000:
            #length of name
            namelen = len(name)   
            
            #avg price per pax
            try:
                dollasigns = rsoup.find_all(class_='drUyy')[0].text
                if "-" in list(dollasigns):
                    first = len(dollasigns.split(' - ')[0])
                    second = len(dollasigns.split(' - ')[-1])
                    price = ((first+second)/2)
                else:
                    price = (len(dollasigns))
            except:
                price = None
                
            #how many photos?
            try:
                if rsoup.find(class_= "details"):
                    photos = rsoup.find(class_= "details").text.split("(")[-1].split(")")[0]
                    if "," in photos:
                        elems = photos.split(',')
                        photos = ""
                        for item in elems:
                            photos += item
                        photos = int(photos)
                    else:
                        photos = int(photos)
            except:
                photos = 0
                
            #reservation available on trip advisor website?
            try: 
                if rsoup.find(class_= 'fBCNs'):
                    canreserve = True
                else:
                    canreserve = False
            except:
                 canreserve = False
            
            #zipcode
            try:
                zipcode = int(rsoup.find_all(class_='fhGHT')[1].text.split(' ')[-2])
            except:
                zipcode = None
            
            #restaurant claimed by business?
            try:
                if rsoup.find(class_='ui_icon verified-checkmark erpsc d'):
                    claimed = True
                elif rsoup.find(class_='fxNHG _S eTpUf'):
                    claimed = False
            except:
                claimed = None
            
            #michelin star?
            if rsoup.find(class_='dpayo PP'):
                michelin = True
            else:
                michelin = False
            
            #travellers choice award?
            if rsoup.find(class_='badge-container tc-badge'):
                travellerschoice = True
            else:
                travellerschoice = False 
            
            #adding restaurant to dataframe
            new_row = {'RestaurantName': name, 'Review': review, 'TARanking' : TAranking, 'NameLength': namelen, 'Avg$PerPax': price, 'NumOfReviews': prev_reviews, 'Photos' : photos, 'ZipCode': zipcode, 'Claimed' : claimed, 'Michelin' : michelin, 'TravellersChoice' : travellerschoice, 'ReserveOnTA' : canreserve}
            if None not in new_row:
                df = df.append(new_row, ignore_index = True)
                print(df.tail(1))
            
            #can't let the website blacklist me!!!
            time.sleep(2)
        
        #next restaurant
        i += 1
        
    #next page of the ranking
    driver.get(ta)
    tripadvisorhttp = 'https://www.tripadvisor.com.sg'
    nexthref = soup.find(class_ = "nav next rndBtn ui_button primary taLnk")["href"]
    nextpage = tripadvisorhttp + nexthref
    driver.get(nextpage)
    ta = nextpage
    soup = bs(driver.page_source, 'html.parser')
    

  RestaurantName  Review TARanking NameLength Avg$PerPax NumOfReviews Photos  \
0  The Courtyard     4.5         1         13          4         1420    863   

  ZipCode Claimed Michelin TravellersChoice ReserveOnTA  
0   49178    True    False             True        True  
        RestaurantName  Review TARanking NameLength Avg$PerPax NumOfReviews  \
1  Entre-Nous creperie     5.0         2         19        2.5         1479   

  Photos ZipCode Claimed Michelin TravellersChoice ReserveOnTA  
1    486  188383    True    False             True       False  
  RestaurantName  Review TARanking NameLength Avg$PerPax NumOfReviews Photos  \
2  Portman's Bar     5.0         3         13          4          248    272   

  ZipCode Claimed Michelin TravellersChoice ReserveOnTA  
2   39594    True    False            False       False  
                RestaurantName  Review TARanking NameLength Avg$PerPax  \
3  Paulaner Bräuhaus Singapore     4.5         4         27        2.5   

  NumOfR

In [6]:
df.head(10)

,RestaurantName,Review,TARanking,NameLength,Avg$PerPax,NumOfReviews,Photos,ZipCode,Claimed,Michelin,TravellersChoice,ReserveOnTA
0,The Courtyard,4.5,1,13,4,1420,863,49178,True,False,True,True
1,Entre-Nous creperie,5.0,2,19,2.5,1479,486,188383,True,False,True,False
2,Portman's Bar,5.0,3,13,4,248,272,39594,True,False,False,False
3,Paulaner Bräuhaus Singapore,4.5,4,27,2.5,1036,473,39596,True,False,True,True
4,Grand Shanghai Restaurant,5.0,5,25,2.5,619,533,169662,True,False,True,False
5,The Rose Veranda,4.5,97,16,2.5,345,345,258350,True,False,True,True
6,Jade,4.5,6,4,4,1030,1290,49178,True,True,True,True
7,Positano Risto,5.0,7,14,2.5,1642,1261,199479,True,False,True,True
8,Bar-Roque Grill,4.5,8,15,2.5,656,461,88539,True,True,True,True
9,Fat Belly Social Steakhouse,5.0,9,27,4,90,343,69620,True,False,False,False


In [10]:
df.to_csv("Desktop/metis_shtuff/sg_restaurants.csv")

In [17]:
#broke down data acquisition into parts
ta = "https://www.tripadvisor.com.sg/Restaurants-g294265-oa540-Singapore.html"

driver = webdriver.Chrome(executable_path='Downloads/chromedriver')
driver.get(ta)
soup = bs(driver.page_source, 'html.parser')

In [18]:
while len(df.index) <= 1000:
    hrefs = soup.find_all(class_='OhCyu')
    i = 0
    while i < len(hrefs):
        #selenium shtuff
        tripadvisorhttp = 'https://www.tripadvisor.com.sg'
        rurl = tripadvisorhttp + hrefs[i].find(href=True)['href']
        driver.get(rurl)
        rsoup = bs(driver.page_source, 'html.parser')
        
        #name of restaurant
        try:
            name = rsoup.find(class_='fHibz').text
        except:
            name = None
        
        #how many previous reviews?
        try:
            prev_reviews = int(rsoup.find(class_='eBTWs').text.split(' ')[0].replace(',',''))
        except:
            prev_reviews = 0
        
        #trip advisor's ranking
        try:
            TAranking = driver.find_elements_by_xpath('.//span[@class = "dyeJW dUpPX"]')[0].text.split("#")[-1].split(" ")[0]
            if "," in list(TAranking):
                TArankinglist = TAranking.split(",")
                TAranking = ""
                for item in TArankinglist:
                    TAranking += item
                TAranking = int(TAranking)
            else:
                TAranking = int(TAranking)
        except:
            TAranking = 0
            
        #the trip advisor review
        try:
            revbubbles = str(rsoup.find(class_="iPqaD _F G- ddFHE eKwUx"))[str(rsoup.find(class_="iPqaD _F G- ddFHE eKwUx")).find('''aria-label="''')+12:str(rsoup.find(class_="iPqaD _F G- ddFHE eKwUx")).find(''' of 5 bubbles"''')+13]
            review = float(str(revbubbles).split(" of")[0])
        except:
            review = None
        
        #only looking at restaurants with more than 5 reviews and a top 1000 ranking
        if name not in df.values and name != None and prev_reviews > 5 and review != None and TAranking <= 2000:
            #length of name
            namelen = len(name)   
            
            #avg price per pax
            try:
                dollasigns = rsoup.find_all(class_='drUyy')[0].text
                if "-" in list(dollasigns):
                    first = len(dollasigns.split(' - ')[0])
                    second = len(dollasigns.split(' - ')[-1])
                    price = ((first+second)/2)
                else:
                    price = (len(dollasigns))
            except:
                price = None
                
            #how many photos?
            try:
                if rsoup.find(class_= "details"):
                    photos = rsoup.find(class_= "details").text.split("(")[-1].split(")")[0]
                    if "," in photos:
                        elems = photos.split(',')
                        photos = ""
                        for item in elems:
                            photos += item
                        photos = int(photos)
                    else:
                        photos = int(photos)
            except:
                photos = 0
                
            #reservation available on trip advisor website?
            try: 
                if rsoup.find(class_= 'fBCNs'):
                    canreserve = True
                else:
                    canreserve = False
            except:
                 canreserve = False
            
            #zipcode
            try:
                zipcode = int(rsoup.find_all(class_='fhGHT')[1].text.split(' ')[-2])
            except:
                zipcode = None
            
            #restaurant claimed by business?
            try:
                if rsoup.find(class_='ui_icon verified-checkmark erpsc d'):
                    claimed = True
                elif rsoup.find(class_='fxNHG _S eTpUf'):
                    claimed = False
            except:
                claimed = None
            
            #michelin star?
            if rsoup.find(class_='dpayo PP'):
                michelin = True
            else:
                michelin = False
            
            #travellers choice award?
            if rsoup.find(class_='badge-container tc-badge'):
                travellerschoice = True
            else:
                travellerschoice = False 
            
            #adding restaurant to dataframe
            new_row = {'RestaurantName': name, 'Review': review, 'TARanking' : TAranking, 'NameLength': namelen, 'Avg$PerPax': price, 'NumOfReviews': prev_reviews, 'Photos' : photos, 'ZipCode': zipcode, 'Claimed' : claimed, 'Michelin' : michelin, 'TravellersChoice' : travellerschoice, 'ReserveOnTA' : canreserve}
            if None not in new_row:
                df = df.append(new_row, ignore_index = True)
                print(df.tail(1))
            
            #can't let the website blacklist me!!!
            time.sleep(2)
        
        #next restaurant
        i += 1
        
    #next page of the ranking
    driver.get(ta)
    tripadvisorhttp = 'https://www.tripadvisor.com.sg'
    nexthref = soup.find(class_ = "nav next rndBtn ui_button primary taLnk")["href"]
    nextpage = tripadvisorhttp + nexthref
    driver.get(nextpage)
    ta = nextpage
    soup = bs(driver.page_source, 'html.parser')
    

          RestaurantName  Review TARanking NameLength Avg$PerPax NumOfReviews  \
972  Katachi Style Sushi     4.5      1427         19        2.5           20   

    Photos ZipCode Claimed Michelin TravellersChoice ReserveOnTA  
972     44  178906    True    False            False        True  
                    RestaurantName  Review TARanking NameLength Avg$PerPax  \
973  Gio - Gourmet Italian Osteria     4.0      1817         29        2.5   

    NumOfReviews Photos ZipCode Claimed Michelin TravellersChoice ReserveOnTA  
973           20     49   88450    True    False            False        True  


NoSuchWindowException: Message: no such window: window was already closed
  (Session info: chrome=97.0.4692.99)


In [16]:
df.to_csv("Desktop/metis_shtuff/1000sgrestaurants.csv")

In [21]:
#broke down data acquisition into parts
ta = "https://www.tripadvisor.com.sg/RestaurantSearch-g294265-oa990-Singapore.html"

driver = webdriver.Chrome(executable_path='Downloads/chromedriver')
driver.get(ta)
soup = bs(driver.page_source, 'html.parser')

In [22]:
while len(df.index) <= 1000:
    hrefs = soup.find_all(class_='OhCyu')
    i = 0
    while i < len(hrefs):
        #selenium shtuff
        tripadvisorhttp = 'https://www.tripadvisor.com.sg'
        rurl = tripadvisorhttp + hrefs[i].find(href=True)['href']
        driver.get(rurl)
        rsoup = bs(driver.page_source, 'html.parser')
        
        #name of restaurant
        try:
            name = rsoup.find(class_='fHibz').text
        except:
            name = None
        
        #how many previous reviews?
        try:
            prev_reviews = int(rsoup.find(class_='eBTWs').text.split(' ')[0].replace(',',''))
        except:
            prev_reviews = 0
        
        #trip advisor's ranking
        try:
            TAranking = driver.find_elements_by_xpath('.//span[@class = "dyeJW dUpPX"]')[0].text.split("#")[-1].split(" ")[0]
            if "," in list(TAranking):
                TArankinglist = TAranking.split(",")
                TAranking = ""
                for item in TArankinglist:
                    TAranking += item
                TAranking = int(TAranking)
            else:
                TAranking = int(TAranking)
        except:
            TAranking = 0
            
        #the trip advisor review
        try:
            revbubbles = str(rsoup.find(class_="iPqaD _F G- ddFHE eKwUx"))[str(rsoup.find(class_="iPqaD _F G- ddFHE eKwUx")).find('''aria-label="''')+12:str(rsoup.find(class_="iPqaD _F G- ddFHE eKwUx")).find(''' of 5 bubbles"''')+13]
            review = float(str(revbubbles).split(" of")[0])
        except:
            review = None
        
        #only looking at restaurants with more than 5 reviews and a top 1000 ranking
        if name not in df.values and name != None and prev_reviews > 5 and review != None and TAranking <= 2000:
            #length of name
            namelen = len(name)   
            
            #avg price per pax
            try:
                dollasigns = rsoup.find_all(class_='drUyy')[0].text
                if "-" in list(dollasigns):
                    first = len(dollasigns.split(' - ')[0])
                    second = len(dollasigns.split(' - ')[-1])
                    price = ((first+second)/2)
                else:
                    price = (len(dollasigns))
            except:
                price = None
                
            #how many photos?
            try:
                if rsoup.find(class_= "details"):
                    photos = rsoup.find(class_= "details").text.split("(")[-1].split(")")[0]
                    if "," in photos:
                        elems = photos.split(',')
                        photos = ""
                        for item in elems:
                            photos += item
                        photos = int(photos)
                    else:
                        photos = int(photos)
            except:
                photos = 0
                
            #reservation available on trip advisor website?
            try: 
                if rsoup.find(class_= 'fBCNs'):
                    canreserve = True
                else:
                    canreserve = False
            except:
                 canreserve = False
            
            #zipcode
            try:
                zipcode = int(rsoup.find_all(class_='fhGHT')[1].text.split(' ')[-2])
            except:
                zipcode = None
            
            #restaurant claimed by business?
            try:
                if rsoup.find(class_='ui_icon verified-checkmark erpsc d'):
                    claimed = True
                elif rsoup.find(class_='fxNHG _S eTpUf'):
                    claimed = False
            except:
                claimed = None
            
            #michelin star?
            if rsoup.find(class_='dpayo PP'):
                michelin = True
            else:
                michelin = False
            
            #travellers choice award?
            if rsoup.find(class_='badge-container tc-badge'):
                travellerschoice = True
            else:
                travellerschoice = False 
            
            #adding restaurant to dataframe
            new_row = {'RestaurantName': name, 'Review': review, 'TARanking' : TAranking, 'NameLength': namelen, 'Avg$PerPax': price, 'NumOfReviews': prev_reviews, 'Photos' : photos, 'ZipCode': zipcode, 'Claimed' : claimed, 'Michelin' : michelin, 'TravellersChoice' : travellerschoice, 'ReserveOnTA' : canreserve}
            if None not in new_row:
                df = df.append(new_row, ignore_index = True)
                print(df.tail(1))
            
            #can't let the website blacklist me!!!
            time.sleep(2)
        
        #next restaurant
        i += 1
        
    #next page of the ranking
    driver.get(ta)
    tripadvisorhttp = 'https://www.tripadvisor.com.sg'
    nexthref = soup.find(class_ = "nav next rndBtn ui_button primary taLnk")["href"]
    nextpage = tripadvisorhttp + nexthref
    driver.get(nextpage)
    ta = nextpage
    soup = bs(driver.page_source, 'html.parser')
    

       RestaurantName  Review TARanking NameLength Avg$PerPax NumOfReviews  \
974  The French Ladle     4.5      1001         16        2.5           49   

    Photos ZipCode Claimed Michelin TravellersChoice ReserveOnTA  
974     54  597626    True    False            False       False  
    RestaurantName  Review TARanking NameLength Avg$PerPax NumOfReviews  \
975       Kafe Utu     4.0      1135          8        2.5           60   

    Photos ZipCode Claimed Michelin TravellersChoice ReserveOnTA  
975    185   89265    True    False            False       False  
                RestaurantName  Review TARanking NameLength Avg$PerPax  \
976  Outram Ya Hua Bak Kut Teh     4.0      1002         25        2.5   

    NumOfReviews Photos ZipCode Claimed Michelin TravellersChoice ReserveOnTA  
976           73     75   89053    True    False            False       False  
         RestaurantName  Review TARanking NameLength Avg$PerPax NumOfReviews  \
977  Tapas,24 Singapore     4.5    

In [23]:
df.to_csv("Desktop/metis_shtuff/dataacquisition.csv")

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1023 entries, 0 to 1022
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   RestaurantName    1023 non-null   object 
 1   Review            1023 non-null   float64
 2   TARanking         1023 non-null   object 
 3   NameLength        1023 non-null   object 
 4   Avg$PerPax        1023 non-null   object 
 5   NumOfReviews      1023 non-null   object 
 6   Photos            1023 non-null   object 
 7   ZipCode           1002 non-null   object 
 8   Claimed           1023 non-null   object 
 9   Michelin          1023 non-null   object 
 10  TravellersChoice  1023 non-null   object 
 11  ReserveOnTA       1023 non-null   object 
dtypes: float64(1), object(11)
memory usage: 96.0+ KB
